In [93]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 80% !important; }</style>"))
pd.set_option("display.max_columns", 65)

In [94]:
from IPython.display import Image

In [95]:
df = pd.read_csv('./datas/final_1.csv')

In [96]:
df1 = df[df.시즌 == 2017]
df2 = df[df.시즌 <= 2016]

In [97]:
def column_new(df):
    # 시즌 카테고리
    df['시즌C'] = df.시즌.apply(lambda x: 0 if x <= 2013 else x)
    # 2014년도에 처음으로 억대 연봉선수 생김
    df['시즌C'] = df.시즌C.apply(lambda x: 1 if 2014 <= x else x)
    #df['시즌C'] = df.시즌C.apply(lambda x: 2 if x > 2014 else x)

    # 경력
    df['경력'] = df['시즌'] - df['데뷔']

    # 이적횟수
    df['이적'] = None
    for x in df['이름']:
        df['이적'][df['이름'] == x] = len(df[df['이름'] == x]['팀'].unique()) - 1
    df.이적 = df.이적.astype(int)

    # 골든 글러브 카테고리
    df.GG1 = df.GG1.apply(lambda x: 4 if x >= 4 else x)

    # 나이 카테고리
    df['나이C'] = df.나이.apply(lambda x: 0 if x <= 29 else x)
    df['나이C'] = df.나이C.apply(lambda x: 1 if 35 >= x > 29 else x)
    df['나이C'] = df.나이C.apply(lambda x: 2 if x >= 36 else x)

    # 출전횟수
    df['횟수'] = 1
    for id_x in df.이름.unique():
        df["횟수"].loc[df[df["이름"] == id_x].reset_index()["index"]] = list(
            df[df["이름"] == id_x].reset_index().reset_index()["level_0"] + 1)
    # 로그연봉
    df['로그연봉'] = np.log(df['연봉'])

In [98]:
column_new(df1)

In [99]:
column_new(df2)

![](./picture/야구1.png)

# 1. 주제

## KBO프로야구 타자 연봉 예측

### 목표
- statiz의 야구선수 데이터를 바탕으로 다음 해의 연봉 예측

### 주제 선정 이유
- KBO 선수 몸값 거품 문제 대두되는 등의 이슈가 생김
- 어떤 기준으로 연봉을 책정하는지에 대해 의문이 생김

# 2. 분석 계획
![](./picture/분석계획1.png)

# 3. Data Preprocessing

![](./picture/시즌기록.png)

![](./picture/연봉.png)

### train/test 데이터 분리
- train(df2) : 시즌 2010 ~ 2017
- test(df1) :  시즌 2018

In [100]:
def season_category():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    fig = make_subplots(
        rows=1, cols=2, shared_xaxes=True, vertical_spacing=0.02,
    )

    a = df2.연봉.groupby([df2.시즌]).mean()
    b = df2.OPS.groupby([df2.시즌]).mean()

    fig.add_trace(go.Bar(x=a.index, y=a, name="시즌별 연봉"), row=1, col=1)
    fig.add_trace(go.Bar(x=b.index, y=b, name="시즌별 OPS"), row=1, col=2)
    fig.update_layout(title_text="시즌별 연봉과 OPS 비교")
    fig.show()

### 시즌 카테고리

In [101]:
season_category()

![](./picture/거품논란.png)

### 나이 카테고리

In [102]:
def age_category():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    fig = make_subplots(shared_xaxes=True, vertical_spacing=0.02,
                        )

    a = df2.연봉.groupby([df2.나이]).median()
    b = df2.OPS.groupby([df2.나이]).mean()

    fig.add_trace(go.Bar(x=a.index, y=a, name="나이별 연봉"), row=1, col=1)

    #fig.add_trace(go.Line(x=b.index, y=b, name="나이별 OPS"), row=1, col=2)

    fig.update_layout(title_text="나이별 연봉")
    fig.show()

In [103]:
age_category()

### 변수에 스케일링을 한 이유

In [104]:
# 변수마다 차이가 커서 scaling을 해줘야 함
def scaling():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    fig = go.Figure()
    fig.add_trace(go.Box(y=df2['G'], name='G'))
    fig.add_trace(go.Box(y=df2['나이'], name='나이'))
    fig.add_trace(go.Box(y=df2['안타'], name='안타'))
    fig.add_trace(go.Box(y=df2['trans'], name='물가'))
    fig.add_trace(go.Box(y=df2['경력'], name='경력'))
    fig.update_layout(title_text="스케일링 해주는 이유")
    fig.show()

In [105]:
scaling()

In [106]:
def for_kor_category():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    kor = df2[df2['용병'] == 0]
    a = kor.연봉.groupby([kor.시즌]).mean()
    fore = df2[df2['용병'] == 1]
    b = fore.연봉.groupby([fore.시즌]).mean()

    fig = go.Figure()

    fig.add_trace(go.Bar(x=a.index, y=a, name="시즌별 한국인  평균 연봉"))
    fig.add_trace(go.Bar(x=b.index, y=b, name="시즌별 용병 평균 연봉"))
    # fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
    # marker_line_width=1.5, opacity=0.6)
    fig.update_layout(title='용병 여부에 따른 평균 연봉')
    fig.show()

In [107]:
def age():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    df4 = df2[df2['용병'] == 0]
    a = df4.연봉[df2.나이 == 25].groupby(df4.경력).mean()
    b = df4.연봉[df2.나이 == 31].groupby(df4.경력).mean()
    fig = go.Figure()

    fig.add_trace(go.Bar(x=a.index, y=a, name="25세 경력 현황"))
    fig.add_trace(go.Bar(x=b.index, y=b, name="31세 경력 현황"))

    fig.update_layout(title='25세, 31세 경력 현황')
    fig.show()

### 필요한 열 추가

In [109]:
def corr():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    fig = make_subplots(
        rows=1, cols=2, vertical_spacing=0.02, shared_yaxes=True)

    heat = df2[['G', '고4', '홈런', 'OPS', '안타', '삼진', 'WAR', '횟수', '연봉']]
    heat1 = df2[['도루',  '도실', '희타', 'BBK', 'WAA', '연봉']]
    fig.add_trace(go.Bar(x=heat.corr()[
                  '연봉'].index[:-1], y=heat.corr()['연봉'].values[:-1], name="상관관계 높음"), row=1, col=1)
    fig.add_trace(go.Bar(x=heat1.corr()['연봉'].index[:-1], y=np.abs(
        heat1.corr()['연봉'].values[:-1]), name="상관관계 낮음"), row=1, col=2)
    fig.update_layout(title_text="연봉과 다른 변수들의 상관관계")
    fig.show()

### 상관관계 분석

In [110]:
corr()

# 4. 예측 모델

## 1. 모델 1

In [117]:
formula1 = '연봉 ~ C(포지션) + C(시즌C)+ C(나이C) + scale(경력) + scale(G) + scale(안타)  +scale(고4) + scale(홈런) + scale(OPS) +scale(횟수) +scale(득점) + scale(삼진)+scale(WAR)'
model1 = sm.OLS.from_formula(formula1, data=df2)
result1 = model1.fit()
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                     연봉   R-squared:                       0.627
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     66.18
Date:                Sat, 18 Apr 2020   Prob (F-statistic):          2.76e-153
Time:                        15:30:26   Log-Likelihood:                -8714.9
No. Observations:                 808   AIC:                         1.747e+04
Df Residuals:                     787   BIC:                         1.757e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      1.374e+04   1577.457      8.713

In [118]:
from sklearn.model_selection import KFold
scores1 = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df2)):
    df_train = df2.iloc[idx_train]
    df_test = df2.iloc[idx_test]
    model1 = sm.OLS.from_formula(formula1, data=df_train)
    result1 = model1.fit()
    pred1 = result1.predict(df_test)
    rss = ((df_test.연봉 - pred1) ** 2).sum()
    tss = ((df_test.연봉 - df_test.연봉.mean()) ** 2).sum()
    rsquared = 1 - rss / tss
    scores1[i] = rsquared
    print("학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(result1.rsquared, rsquared))

학습 R2 = 0.61006265, 검증 R2 = 0.66333192
학습 R2 = 0.64297331, 검증 R2 = 0.49758571
학습 R2 = 0.63180388, 검증 R2 = 0.58876862
학습 R2 = 0.64698943, 검증 R2 = 0.54995789
학습 R2 = 0.62119049, 검증 R2 = 0.62769494


In [119]:
def model1():
    import plotly.graph_objects as go
    import pandas as pd

    pred1 = result1.predict(df2)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df2['연봉'], y=pred1,
                             mode='markers', marker_color=df2['로그연봉']))

    fig.update_layout(title='model 1 : 야구선수 연봉 예측')
    fig.show()

In [120]:
model1()

## 2. 모델 2
- 연봉에 로그를 취함

In [203]:
df2['로그연봉1'] = 4*np.log(df2['연봉'])

In [212]:
formula2 = '로그연봉~ C(포지션) + C(시즌C)+ C(나이C) + scale(경력) + scale(G) + scale(안타)  +scale(고4) + scale(홈런) +scale(이적) + scale(OPS) +scale(횟수) +scale(득점) + scale(삼진)+scale(WAR)'
model2 = sm.OLS.from_formula(formula2, data=df2)
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                   로그연봉   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.805
Method:                 Least Squares   F-statistic:                     160.0
Date:                Sat, 18 Apr 2020   Prob (F-statistic):          9.27e-267
Time:                        15:39:47   Log-Likelihood:                -425.58
No. Observations:                 808   AIC:                             895.2
Df Residuals:                     786   BIC:                             998.4
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         9.1084      0.055    164.517

In [213]:
from sklearn.model_selection import KFold

scores2 = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=13)
for i, (idx_train, idx_test) in enumerate(cv.split(df2)):
    df_train = df2.iloc[idx_train]
    df_test = df2.iloc[idx_test]
    model2 = sm.OLS.from_formula(formula2, data=df_train)
    result2 = model2.fit()
    pred2 = result2.predict(df_test)
    rss = ((df_test.로그연봉 - pred2) ** 2).sum()
    tss = ((df_test.로그연봉 - df_test.로그연봉.mean()) ** 2).sum()
    rsquared = 1 - rss / tss
    scores2[i] = rsquared
    print("학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(result2.rsquared, rsquared))

학습 R2 = 0.80934157, 검증 R2 = 0.81081850
학습 R2 = 0.80537396, 검증 R2 = 0.82106362
학습 R2 = 0.82281152, 검증 R2 = 0.74121284
학습 R2 = 0.80929885, 검증 R2 = 0.80600425
학습 R2 = 0.80926787, 검증 R2 = 0.81254335


In [217]:
def model2():
    import plotly.graph_objects as go
    import pandas as pd

    pred2 = result2.predict(df2)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df2['로그연봉'], y=pred2,
                             mode='markers', marker_color=df2['로그연봉']))
    fig.update_layout(title='야구선수 연봉 예측')
    fig.show()

In [218]:
model2()

![](./picture/용병아웃라이어.png)

## 3. 모델 3
 - 높은 연봉을 받는 선수들을 잘 예측하지 못함
     - 수상 지표 추가(크롤링)
     - 팀 순위 추가 (크롤링)
     - 용병 여부 추가 (크롤링)

![](./picture/수상.png)

![](./picture/팀순위.png)

In [126]:
def prize():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    fig = go.Figure()

    a = df2.연봉.groupby([df2.수상N1]).mean()

    fig.add_trace(go.Bar(x=a.index, y=a, name="수상 누적 횟수별 연봉"))

    fig.update_layout(title_text="수상 누적 횟수와 종류별 연봉")
    fig.show()

In [127]:
def MVP():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    fig = go.Figure()
    df2['MVP'] = df2['MVP'].apply(lambda x: '시즌MVP' if x == 1 else 0)
    c = df2.연봉.groupby([df2.MVP]).mean()
    b = df2.연봉.groupby([df2.수상]).mean()
    fig.add_trace(go.Bar(x=b.index, y=b, name="수상종류별 연봉"))
    fig.add_trace(go.Bar(x=c.index, y=c, name="시즌 MVP 수상 여부별 연봉"))
    fig.show()

In [128]:
prize()

In [129]:
MVP()

In [130]:
def team_rank():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    fig = go.Figure()
    df4 = df2[df2['시즌C'] == 0]
    temp = df4[df4['팀'] == 'SK'].groupby(df4['순위']).mean()
    fig.add_trace(go.Bar(x=temp["순위"], y=temp["연봉"], name='SK'))
    temp1 = df4[df4['팀'] == '삼성'].groupby(df4['순위']).mean()
    fig.add_trace(go.Bar(x=temp1["순위"], y=temp1["연봉"], name='삼성'))
    temp2 = df4[df4['팀'] == '두산'].groupby(df4['순위']).mean()
    fig.add_trace(go.Bar(x=temp2["순위"], y=temp2["연봉"], name='두산'))
    temp3 = df4[df4['팀'] == '키움'].groupby(df4['순위']).mean()
    fig.add_trace(go.Bar(x=temp3["순위"], y=temp3["연봉"], name='키움'))
    temp4 = df4[df4['팀'] == 'LG'].groupby(df4['순위']).mean()
    fig.add_trace(go.Bar(x=temp4["순위"], y=temp4["연봉"], name='LG'))
    temp5 = df4[df4['팀'] == 'NC'].groupby(df4['순위']).mean()
    fig.add_trace(go.Bar(x=temp5["순위"], y=temp5["연봉"], name='NC'))

    fig.update_layout(title_text="팀당 순위별 연봉 변화")
    fig.show()

In [131]:
team_rank()

### 용병 여부

![](./picture/용병1.png)

In [132]:
for_kor_category()

In [133]:
def team_rank1():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    fig = go.Figure()
    df4 = df2[df2['시즌C'] >= 1.0]
    temp = df4[df4['팀'] == 'SK'].groupby(df4['순위']).mean()
    fig.add_trace(go.Bar(x=temp["순위"], y=temp["연봉"], name='SK'))
    temp1 = df4[df4['팀'] == '삼성'].groupby(df4['순위']).mean()
    fig.add_trace(go.Bar(x=temp1["순위"], y=temp1["연봉"], name='삼성'))
    temp2 = df4[df4['팀'] == '두산'].groupby(df4['순위']).mean()
    fig.add_trace(go.Bar(x=temp2["순위"], y=temp2["연봉"], name='두산'))
    temp3 = df4[df4['팀'] == '키움'].groupby(df4['순위']).mean()
    fig.add_trace(go.Bar(x=temp3["순위"], y=temp3["연봉"], name='키움'))
    temp4 = df4[df4['팀'] == 'LG'].groupby(df4['순위']).mean()
    fig.add_trace(go.Bar(x=temp4["순위"], y=temp4["연봉"], name='LG'))
    temp5 = df4[df4['팀'] == 'NC'].groupby(df4['순위']).mean()
    fig.add_trace(go.Bar(x=temp5["순위"], y=temp5["연봉"], name='NC'))

    fig.update_layout(title_text="2014년 이후 팀당 순위별 연봉 변화")
    fig.show()

In [134]:
def position_mvp():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    mvp = df2[df2['MVP1'] == 0]
    mvp1 = df2[df2['MVP1'] == 1]

    a = mvp.연봉.groupby([mvp.포지션]).mean()
    b = mvp1.연봉.groupby([mvp1.포지션]).mean()

    fig = go.Figure()

    fig.add_trace(go.Bar(x=a.index, y=a, name="포지션별 연봉"))
    fig.add_trace(go.Bar(x=b.index, y=b, name="시즌 MVP를 수상 연봉"))

    fig.update_layout(title='포지션별 시즌MVP 수상 여부에 따른 연봉')
    fig.show()

In [135]:
team_rank1()

In [136]:
position_mvp()

In [137]:
def GG_안타():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    fig = go.Figure()
    ngg = df2[df2.GG1 == 0]

    df2['안타1'] = None
    df2['안타1'] = df2.안타.apply(lambda x: 0 if x <= 17 else x)
    df2['안타1'] = df2.안타1.apply(lambda x: 17 if 56 >= x > 17 else x)
    df2['안타1'] = df2.안타1.apply(lambda x: 56 if 105 >= x > 56 else x)
    df2['안타1'] = df2.안타1.apply(lambda x: 105 if x > 105 else x)

    df2['안타1'] = df2.안타1.apply(lambda x: '0~17' if x == 0 else x)
    df2['안타1'] = df2.안타1.apply(lambda x: '17~56' if x == 17 else x)
    df2['안타1'] = df2.안타1.apply(lambda x: '56~105' if x == 56 else x)
    df2['안타1'] = df2.안타1.apply(lambda x: '105~201' if x == 105 else x)

    temp = df2[df2['GG1'] == 0].groupby(df2['안타1']).mean()
    temp.sort_values('로그연봉', inplace=True)
    fig.add_trace(go.Bar(x=temp.index, y=temp["연봉"], name='GG 수상하지 않은 경우'))
    temp1 = df2[df2['GG1'] >= 1].groupby(df2['안타1']).mean()
    fig.add_trace(go.Bar(x=temp1.index, y=temp1["연봉"], name='GG 수상한 경우'))

    fig.update_layout(title_text="안타와 GG 상호작용 이유")
    fig.show()

In [138]:
GG_안타()

In [139]:
df2['MVP'] = df2['MVP'].apply(lambda x: 1 if x == '시즌MVP' else 0)

In [141]:
formula3 = '로그연봉 ~ C(포지션) + C(시즌C)+ C(나이C) + scale(경력) + scale(G)   +scale(고4) + scale(홈런) + C(용병) +scale(이적) + scale(OPS) +scale(횟수) +scale(득점)  + scale(삼진)+scale(WAR) + scale(수상N) + scale(수상N1) + scale(WAR)+ scale(ALMVP) +scale(MVP)+ C(팀):scale(순위) + C(GG):scale(안타)'
model3 = sm.OLS.from_formula(formula3, data=df2)
result3 = model3.fit()
print(result3.summary())

                            OLS Regression Results                            
Dep. Variable:                   로그연봉   R-squared:                       0.846
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     114.8
Date:                Sat, 18 Apr 2020   Prob (F-statistic):          4.38e-285
Time:                        15:31:02   Log-Likelihood:                -340.27
No. Observations:                 808   AIC:                             756.5
Df Residuals:                     770   BIC:                             934.9
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                9.0927 

In [142]:
from sklearn.model_selection import KFold
scores3 = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=10)
for i, (idx_train, idx_test) in enumerate(cv.split(df2)):
    df_train = df2.iloc[idx_train]
    df_test = df2.iloc[idx_test]
    model3 = sm.OLS.from_formula(formula3, data=df_train)
    result3 = model3.fit()
    pred3 = result3.predict(df_test)
    rss = ((df_test.로그연봉 - pred3) ** 2).sum()
    tss = ((df_test.로그연봉 - df_test.로그연봉.mean()) ** 2).sum()
    rsquared = 1 - rss / tss
    scores3[i] = rsquared
    print("학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(result3.rsquared, rsquared))

학습 R2 = 0.84755537, 검증 R2 = 0.82847429
학습 R2 = 0.83916188, 검증 R2 = 0.86899760
학습 R2 = 0.84416108, 검증 R2 = 0.84743848
학습 R2 = 0.85351730, 검증 R2 = 0.80785153
학습 R2 = 0.85697908, 검증 R2 = 0.79893005


In [143]:
def model3():
    import plotly.graph_objects as go
    import pandas as pd
    model3 = sm.OLS.from_formula(formula3, data=df2)
    result3 = model3.fit()
    pred3 = result3.predict(df2)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df2['로그연봉'], y=pred3,
                             mode='markers', marker_color=df2['로그연봉']))

    fig.add_trace(go.Scatter(x=[7.5, 10, 12], y=[7.5, 10, 12]))
    fig.update_layout(title='model3 야구선수 연봉 예측')
    fig.show()

In [144]:
def model23():
    import plotly.graph_objects as go
    import pandas as pd
    model3 = sm.OLS.from_formula(formula3, data=df2)
    result3 = model3.fit()
    pred3 = result3.predict(df2)
    fig = go.Figure()

    pred2 = result2.predict(df2)
    fig.add_trace(go.Scatter(x=df2['로그연봉'], y=pred2,
                             mode='markers', name='model2'))
    fig.add_trace(go.Scatter(x=df2['로그연봉'], y=pred3,
                             mode='markers', name='model3'))

    fig.add_trace(go.Scatter(x=[7.5, 10, 12], y=[7.5, 10, 12]))
    fig.update_layout(title='model2와 model3 비교')
    fig.show()

In [145]:
model3()

In [146]:
model23()

## 4. 모델 4
 - FA 계약 추가(카테고리)
    - FA 계약 체결 당해년도
    - FA 계약중
    - FA 계약 X
- FA 계약중인 선수는 이전년도의 연봉과 같아서 데이터를 분리한뒤
    - 계약중이 아닌 데이터로 학습 -> 예측
    - 계약중인 데이터는 이전년도의 연봉을 가져옴
    
- 현재가치 반영(trans)

In [147]:
df3 = df2[df2['계약'] == '계약중']

In [148]:
formula4 = '로그연봉 ~ C(포지션) + C(시즌C)+ C(나이C)+scale(경력) + scale(G) +scale(고4) + scale(홈런) + C(용병) +scale(이적) + scale(OPS) +scale(횟수) +scale(득점)  + scale(삼진)+ scale(WAR) + scale(수상N) + scale(수상N1) + scale(WAR)+ scale(ALMVP) +scale(MVP)+ C(팀):scale(순위) + C(GG):scale(안타) + scale(trans)+C(계약)'
model4 = sm.OLS.from_formula(formula4, data=df2)
result4 = model4.fit()
print(result4.summary())

                            OLS Regression Results                            
Dep. Variable:                   로그연봉   R-squared:                       0.868
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     126.4
Date:                Sat, 18 Apr 2020   Prob (F-statistic):          2.70e-307
Time:                        15:31:08   Log-Likelihood:                -278.42
No. Observations:                 808   AIC:                             638.8
Df Residuals:                     767   BIC:                             831.3
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                9.0780 

![](./datas/model4.png)

In [149]:
from sklearn.model_selection import KFold
scores4 = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=3)
for i, (idx_train, idx_test) in enumerate(cv.split(df2)):
    df_train = df2.iloc[idx_train]
    df_test = df2.iloc[idx_test]

    df_train = df_train[df_train["계약"] != "계약중"]
    df_test_FA = df_test[df_test["계약"] == "계약중"]['로그연봉']
    df_test_t = df_test[df_test["계약"] != "계약중"]

    model4 = sm.OLS.from_formula(formula4, data=df_train)
    result4 = model4.fit()
    pred4 = result4.predict(df_test_t)

    rss = ((df_test.로그연봉 - df_test_FA.append(pred4)) ** 2).sum()
    tss = ((df_test.로그연봉 - df_test.로그연봉.mean()) ** 2).sum()
    rsquared = 1 - rss / tss
    scores4[i] = rsquared
    print("학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(result4.rsquared, rsquared))

학습 R2 = 0.86203864, 검증 R2 = 0.87855808
학습 R2 = 0.86166936, 검증 R2 = 0.86614750
학습 R2 = 0.86428382, 검증 R2 = 0.86577885
학습 R2 = 0.85683736, 검증 R2 = 0.88987136
학습 R2 = 0.86709572, 검증 R2 = 0.85194200


In [150]:
df3 = df2[df2['계약'] != '계약중']


def model4():
    import plotly.graph_objects as go
    import pandas as pd
    model4 = sm.OLS.from_formula(formula4, data=df3)
    result4 = model4.fit()
    pred4 = result4.predict(df3)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df3['로그연봉'], y=pred4,
                             mode='markers', marker_color=df3['로그연봉']))
    fig.add_trace(go.Scatter(x=[7.5, 10, 12], y=[7.5, 10, 12]))
    fig.update_layout(title='model4 야구선수 연봉 예측')
    fig.show()

In [151]:
model4()

In [152]:
def scores():
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    scores = [scores1, scores2, scores3, scores4]
    fig = go.Figure()
    fig.add_trace(go.Box(y=scores1, name='model1'))
    fig.add_trace(go.Box(y=scores2, name='model2'))
    fig.add_trace(go.Box(y=scores3, name='model3'))
    fig.add_trace(go.Box(y=scores4, name='model4'))

    fig.update_layout(title_text="모델별 성능")
    fig.show()

### 모델별 성능 비교

In [153]:
scores()

# 5. test 데이터로 검증

In [55]:
df1 = df1.reset_index(drop=True)
df1X = df1.drop(['연봉', '로그연봉'], axis=1)
df1y = df1['로그연봉']

df1A = df1[df1.계약 == '계약중']['로그연봉']
df1B = df1X[df1X.계약 != '계약중']
pred = result4.predict(df1B)

In [56]:
df1 = df1.reset_index(drop=True)
df1X = df1.drop(['연봉', '로그연봉'], axis=1)
df1y = df1['로그연봉']

df1A = df1[df1.계약 == '계약중']['로그연봉']
df1B = df1X[df1X.계약 != '계약중']
pred = result4.predict(df1B)

yhat = pd.concat([pred, df1A]).sort_index()

In [57]:
mse = sk.metrics.mean_squared_error(
    df1y, yhat, sample_weight=None, multioutput='uniform_average')
mae = sk.metrics.mean_absolute_error(
    df1y, yhat, sample_weight=None, multioutput='uniform_average')
r2 = sk.metrics.r2_score(df1y, yhat, sample_weight=None,
                         multioutput='uniform_average')
mse, mae, r2

(0.13054645213320462, 0.24052298993882965, 0.8732424610549171)

In [58]:
np.mean(np.absolute(np.exp(df1y) - np.exp(yhat)))

4335.493419465774

In [59]:
def test():
    import plotly.graph_objects as go

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df1['로그연봉'], y=yhat,
                             mode='markers', marker_color=df1['로그연봉']))
    fig.add_trace(go.Scatter(x=[7.5, 10, 12], y=[7.5, 10, 12]))
    fig.update_layout(title='test 야구선수 연봉 예측 결과')
    fig.show()

In [86]:
test()

### 프로젝트 회고
 - 연봉이 높은 선수들을 잘 예측하지 못함
     - 이후 RandomForest와 부스팅 기법을 이용해서 분석을 시도할 예정
     
 - 2019년도 연봉이 아직 올라오지 않아 추후 2019년도 연봉도 예측할 예정

## Lasso
- 가중치가 작고, 성능의 변화가 크게 없으므로 정규화를 하지 않아도 됨

In [206]:
from sklearn.model_selection import validation_curve
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

alphas = np.logspace(-10, -2, 30)
# model.fit_regularized(alpha=0.01, L1_wt=0)
train_scores = []
test_scores = []
for alpha in alphas:
    test_ls = []
    train_ls = []
    #test_score = np.mean(result, X, y, scoring='neg_mean_squared_error', cv=5)
    cv = KFold(5, shuffle=True, random_state=2)
    for i, (idx_train, idx_test) in enumerate(cv.split(df3)):
        df_train = df3.iloc[idx_train]
        df_test = df3.iloc[idx_test]

        model = sm.OLS.from_formula(formula4, data=df_train)
        result = model.fit_regularized(alpha=alpha, L1_wt=1)
        pred = result.predict(df_train)
        train_ls.append(-mean_squared_error(df_train.로그연봉, pred))

        pred = result.predict(df_test)
        test_ls.append(-mean_squared_error(df_test.로그연봉, pred))
    train_scores.append(np.mean(train_ls))
    test_scores.append(np.mean(test_ls))
    print(alpha)

KeyboardInterrupt: 

In [ ]:
def lasso():
    optimal_alpha = alphas[np.argmax(test_scores)]
    optimal_score = np.max(test_scores)

    plt.semilogx(alphas, test_scores, '-', label="검증성능")
    plt.semilogx(alphas, train_scores, '--', label="학습성능"),
    plt.axhline(optimal_score, linestyle=':')
    plt.axvline(optimal_alpha, linestyle=':')
    plt.scatter(optimal_alpha, optimal_score)
    plt.ylabel('성능')
    plt.xlabel('정규화 가중치')
    plt.legend()
    plt.show()

In [ ]:
lasso()